<a href="https://colab.research.google.com/github/aman-p7952/NoOfProduct/blob/main/NoOfProduct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Packages

In [ ]:
import shutil
import os
import cv2
import shutil
import torch
from IPython.display import Image 
import json

# Downloading data and yolov5 model

In [ ]:

!wget https://github.com/gulvarol/grocerydataset/releases/download/1.0/GroceryDataset_part1.tar.gz
!tar xzvf GroceryDataset_part1.tar.gz

shutil.rmtree('/content/ShelfImages') #removing ShelfImages folder
!wget https://storage.googleapis.com/open_source_datasets/ShelfImages.tar.gz
!wget https://github.com/gulvarol/grocerydataset/releases/download/1.0/GroceryDataset_part2.tar.gz
!tar xzvf GroceryDataset_part2.tar.gz
!tar xzvf ShelfImages.tar.gz
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies
%cd /content/yolov5

Streaming output truncated to the last 5000 lines.
ProductImagesFromShelves/2/C3_P03_N1_S5_1.JPG_484_124_160_216.png
BrandImagesFromShelves/1/C1_P11_N1_S5_1.JPG_1776_360_192_244.png
BrandImagesFromShelves/7/C2_P01_N1_S5_2.JPG_692_1316_116_152.png
ProductImagesFromShelves/2/C4_P08_N1_S2_1.JPG_1704_1308_256_344.png
ProductImagesFromShelves/0/C3_P01_N2_S3_1.JPG_1260_1856_260_356.png
BrandImagesFromShelves/4/C2_P08_N1_S6_2.JPG_144_416_116_168.png
ProductImagesFromShelves/0/C4_P07_N1_S4_1.JPG_1088_44_152_316.png
BrandImagesFromShelves/2/C3_P03_N1_S3_1.JPG_1592_1092_280_376.png
ProductImagesFromShelves/2/C4_P01_N3_S3_1.JPG_40_388_196_272.png
ProductImagesFromShelves/0/C3_P07_N4_S2_1.JPG_3436_500_388_564.png
ProductImagesFromShelves/0/C1_P10_N2_S3_1.JPG_916_240_228_324.png
ProductImagesFromShelves/0/C1_P03_N3_S2_1.JPG_2736_584_216_284.png
ProductImagesFromShelves/0/C2_P01_N2_S3_2.JPG_1604_1248_140_204.png
ProductImagesFromShelves/2/C3_P03_N1_S3_1.JPG_1016_1088_280_376.png
ProductImagesFromShe

/content/yolov5


# Data prepration for yolo

In [ ]:

#shutil.rmtree('/content/ShelfImages/labels')
!mkdir -p /content/datasets/ShelfImages/labels/train
!mkdir -p /content/datasets/ShelfImages/labels/test
!mkdir -p /content/datasets/ShelfImages/images/
!mv /content/ShelfImages/train /content/datasets/ShelfImages/images/ 
!mv /content/ShelfImages/test /content/datasets/ShelfImages/images/ 
name_list_test=os.listdir('/content/datasets/ShelfImages/images/test/')
name_list_train=os.listdir('/content/datasets/ShelfImages/images/train/')
name_list=name_list_test+name_list_train
#os.chdir('/content/ShelfImages/labels/')
for n in range(len(name_list)):
  im=cv2.imread("/content/datasets/ShelfImages/images/test/"+name_list[n]) if n<len(name_list_test) else cv2.imread("/content/datasets/ShelfImages/images/train/"+name_list[n]) 

  shape=im.shape
  f1  = open('/content/datasets/ShelfImages/labels/test/'+name_list[n][:-4]+".txt", 'a+')
  f2  = open('/content/datasets/ShelfImages/labels/train/'+name_list[n][:-4]+".txt", 'a+')
  f=f1 if n<len(name_list_test) else f2 
  for i in range(11):
    productname_list=os.listdir('/content/ProductImagesFromShelves/'+str(i))
    for j in range(len(productname_list)):
      a =productname_list[j].split('.JPG')
      if a[0]+'.JPG' == name_list[n]:
        b=a[1][1:-4].split('_')
        xywh=[int(i) for i in b]
        xywh[0]/=shape[1]
        xywh[2]/=shape[1]
        xywh[1]/=shape[0]
        xywh[3]/=shape[0]
        xywh[0]+=xywh[2]/2
        xywh[1]+=xywh[3]/2
        f.write(str(i)+" {:.06f} {:.06f} {:.06f} {:.06f}\n".format(xywh[0],xywh[1],xywh[2],xywh[3]))
  f1.close()
  f2.close()

# Training

In [ ]:
!python train.py --img 640 --batch 32 --epochs 30 --data shelf.yaml --weights yolov5s.pt --single-cls

train: weights=yolov5s.pt, cfg=, data=shelf.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=True, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-500-g276b674 torch 1.9.1+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, she

# Detection on Test Data

In [ ]:
!python detect.py --source '../datasets/ShelfImages/images/test/' --weights runs/train/exp/weights/best.pt --conf-thres 0.5

detect: weights=['runs/train/exp/weights/best.pt'], source=../datasets/ShelfImages/images/test/, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-500-g276b674 torch 1.9.1+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
image 1/71 /content/datasets/ShelfImages/images/test/C1_P02_N1_S5_1.JPG: 608x640 48 items, Done. (0.022s)
image 2/71 /content/datasets/ShelfImages/images/test/C1_P02_N2_S2_1.JPG: 480x640 28 items, Done. (0.021s)
image 3/71 /content/datasets/ShelfImages/images/test/C1_P02_N2_S3_1.JPG: 480x640 30 items, Done. (0.019s)
image 4/71 /content/datasets/ShelfImages/images/test/C1_P03_N1_S2_1.JPG

# Validation on test data

In [ ]:
!python val.py --data shelf.yaml  --weights runs/train/exp/weights/best.pt --img 640 --save-txt --single-cls --save-conf


val: data=/content/yolov5/data/shelf.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=True, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False
YOLOv5 🚀 v5.0-500-g276b674 torch 1.9.1+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
val: Scanning '/content/datasets/ShelfImages/labels/test.cache' images and labels... 71 found, 0 missing, 0 empty, 0 corrupted: 100% 71/71 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 3/3 [00:37<00:00, 12.43s/it]
                 all         71       2648      0.998      0.863      0.902      0.624
Speed: 0.6ms pre-process, 23.4ms inference, 16.1ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs/val/exp2
71 labels

# saving detected images

In [ ]:
#!ls /content/drive/MyDrive/Product_Detection_using_Yolov5
!mv /content/yolov5/runs/detect/exp /content/drive/MyDrive/Product_Detection_using_Yolov5/ 

# Creating image2products.json

In [ ]:
CONF_THRE=0.5
image2products={}
image_name = os.listdir('/content/yolov5/runs/val/exp/labels')
for name in image_name:
  f=open('/content/yolov5/runs/val/exp/labels/'+name,"r")
  lines=f.readlines()
  val=0
  for i in range(len(lines)): #counting no. of products
    conf=float(lines[i].split(' ')[5]) 
    if conf>=CONF_THRE:
      val+=1
  image2products[name[:-4]]=val
  f.close()
with open("image2products.json", "w") as f:
    json.dump(image2products, f)
